In [4]:
!pip install python-dotenv

In [1]:
import os
from dotenv import load_dotenv

env_available = load_dotenv() # loads the environment variables from .env file
assert env_available, "No .env file found"

# Broadly still following the tutorial from Educative, but the llama index api is now completely different, so we have to adapt. 

In [2]:
from pydantic import BaseModel
from typing import Optional, List

class WikiPageList(BaseModel):
    """Data model for list of wiki pages"""
    pages: List[str]

In [3]:
from llama_index.program.openai import OpenAIPydanticProgram
def wikipage_list(query):
    """This function extracts the Wikipedia pages from the query using OpenAI's GPT-3 API and returns them as a list (but stored in a WikiPageList object). 
    Args: 
        query (str): The query string containing the Wikipedia pages to extract. It must start with `please index:` followed by the Wikipedia pages.
    Returns:
        WikiPageList: A data model object containing the list of Wikipedia pages extracted from the query.
    """
    openai_key = os.getenv("OPENAI_API_KEY")
    if not openai_key:
        raise ValueError("No OpenAI API key found")
    
    prompt_template_str = """
    Given the input {query}, extract the Wikipedia pages mentioned after "please index:" and return them as a list.
    If only one page is mentioned, return a single element list.
    """

    program = OpenAIPydanticProgram.from_defaults(
        output_cls=WikiPageList,
        prompt_template_str=prompt_template_str,
        verbose=True,
    )

    wikipage_requests = program(query=query)
    return wikipage_requests

wikipage_list("please index: Jesus Christ, Buddha, and Muhammad.")


Function call: WikiPageList with args: {"pages":["Jesus Christ","Buddha","Muhammad"]}


WikiPageList(pages=['Jesus Christ', 'Buddha', 'Muhammad'])

In [18]:
wikipage_list("please index: Alexander Grothendieck, Alan Turing, Gottlieb Frege, and David Hilbert.")

Function call: WikiPageList with args: {"pages":["Alexander Grothendieck","Alan Turing","Gottlieb Frege","David Hilbert"]}


WikiPageList(pages=['Alexander Grothendieck', 'Alan Turing', 'Gottlieb Frege', 'David Hilbert'])

# Create the documents function

In [7]:
# There are dedicate readers on LlamaHub. 
!pip install llama-index-readers-wikipedia
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=4c179c967466df3a152ed542c6deaa52936b813b3c93e69f2b07e2eec3a912f9
  Stored in directory: /Users/dominicculver/Library/Caches/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [8]:
from llama_index.readers.wikipedia import WikipediaReader

reader = WikipediaReader()